### importing require packages

In [1]:
from __future__ import print_function

import json
import os
import numpy as np
import sys
import h5py

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from keras.engine import Input
from keras.layers import Embedding, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.preprocessing import sequence
from embeddings import Embeddings
from keras.callbacks import ModelCheckpoint

from nltk.tokenize import word_tokenize
import random

Using TensorFlow backend.


## Instantiate Embeddings 

In [2]:
embeddings = Embeddings(100, 4, 1, 4)

### getting data from preprocessing

In [3]:
word2vec_weights = embeddings.get_weights()
word2index, index2word = embeddings.get_vocabulary()
word2vec_model = embeddings.get_model()
tokenized_indexed_sentences = embeddings.get_tokenized_indexed_sentences()

### generating training data

In [4]:
window_size = 3
vocab_size = len(word2index)
print(vocab_size)
#sorted(window_size,reverse=True)
#sentence_max_length = max([len(sentence) for sentence in tokenized_indexed_sentence ])

170760


In [5]:
seq_in = []
seq_out = []
# generating dataset
for sentence in tokenized_indexed_sentences:
    for i in range(len(sentence)-window_size-1):
        x = sentence[i:i + window_size]
        y = sentence[i + window_size]
        seq_in.append(x)#[]
        seq_out.append(word2vec_weights[y])

# converting seq_in and seq_out into numpy array
seq_in = np.array(seq_in)
seq_out = np.array(seq_out)
n_samples = len(seq_in)
print ("Number of samples : ", n_samples)

Number of samples :  3381370


## Defining model

In [6]:
# Changes to the model to be done here
model = Sequential()
model.add(Embedding(input_dim=word2vec_weights.shape[0], output_dim=word2vec_weights.shape[1], weights=[word2vec_weights]))
model.add(LSTM(512,return_sequences =True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dense(word2vec_weights.shape[1], activation='relu'))
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         17076000  
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 512)         1255424   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 100)               51300     
Total params: 20,481,924
Trainable params: 20,481,924
Non-trainable params: 0
_________________________________________________________________


In [7]:
model_weights_path = "../weights/lstm-2-256-batchsize-128-epochs-10"
if not os.path.exists(model_weights_path):
    os.makedirs(model_weights_path)
checkpoint_path = model_weights_path + '/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

## Train Model

In [9]:
model.fit(seq_in[:1000], seq_out[:1000], epochs=10, verbose=1, validation_split=0.2, batch_size=128, callbacks=[checkpoint])

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 4s - loss: 1.1103 - acc: 0.1237 - val_loss: 0.7468 - val_acc: 0.1150
Epoch 2/10
800/800 [==============================] - 5s - loss: 1.0945 - acc: 0.2012 - val_loss: 0.7360 - val_acc: 0.0650
Epoch 3/10
800/800 [==============================] - 4s - loss: 1.0879 - acc: 0.2012 - val_loss: 0.7438 - val_acc: 0.1000
Epoch 4/10
800/800 [==============================] - 4s - loss: 1.0818 - acc: 0.1787 - val_loss: 0.7465 - val_acc: 0.1050
Epoch 5/10
800/800 [==============================] - 4s - loss: 1.0766 - acc: 0.2025 - val_loss: 0.7569 - val_acc: 0.0700
Epoch 6/10
800/800 [==============================] - 4s - loss: 1.0715 - acc: 0.2038 - val_loss: 0.7534 - val_acc: 0.0650
Epoch 7/10
800/800 [==============================] - 4s - loss: 1.0652 - acc: 0.1862 - val_loss: 0.7734 - val_acc: 0.0900
Epoch 8/10
800/800 [==============================] - 4s - loss: 1.0575 - acc: 0.2038 - val_l

### model predict

In [ ]:
start = 97
pattern = list(seq_in[start])
print("\"",' '.join(index2word[index] for index in pattern))
for i in range(10):
    prediction = model.predict(np.array([pattern]))
    pred_word = word2vec_model.similar_by_vector(prediction[0])[0][0]
    sys.stdout.write(pred_word+" ")
    pattern.append(word2index[pred_word])
    pattern = pattern[1:len(pattern)]

## Accuracy

In [ ]:
def accuracy(no_of_preds):
    correct=0
    wrong=0
    random.seed(1)
    for i in random.sample(range(0, seq_in.shape[0]), no_of_preds):
        start = i
        sentence = list(seq_in[start])
        prediction = model.predict(np.array([sentence]))
        pred_word = word2vec_model.similar_by_vector(prediction[0])[0][0]
        next_word_index = list(seq_in[start+1])
        next_word = index2word[next_word_index[-1]]
        sim = word2vec_model.similarity(pred_word,next_word)
        if (sim >= 0.6):
            correct +=1
        else : wrong +=1
    print('correct: '+str(correct)+(' wrong: ')+str(wrong))
    accur = float(correct/(correct+wrong))
    print('accuracy = ',float(accur))
    

In [ ]:
# n = no. of predictions
accuracy(400)